# RAG pipeline using Open Source Large Languages

Chat with Website using Zephyr 7B model

## Installation

In [ ]:
!pip install langchain faiss-cpu sentence-transformers chromadb

## Import RAG components required to build pipeline

In [ ]:
from langchain.llms import HuggingFaceHub
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.vectorstores import FAISS, Chroma
from langchain.chains import RetrievalQA, LLMChain

In [ ]:
import os
from getpass import getpass

HF_TOKEN = getpass("HF Token:")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HF_TOKEN

HF Token:··········


## External data/document - ETL

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
WEBSITE_URL = "https://python.langchain.com/docs/get_started/introduction"

In [ ]:
loader = WebBaseLoader(WEBSITE_URL)
loader.requests_per_second = 1
docs = loader.aload()

Fetching pages: 100%|##########| 1/1 [00:00<00:00,  7.74it/s]


In [ ]:
docs

[Document(page_content="\n\n\n\n\nIntroduction | 🦜️🔗 Langchain\n\n\n\n\n\n\n\nSkip to main contentDocsUse casesIntegrationsGuidesAPIMorePeopleVersioningChangelogContributingTemplatesCookbooksTutorialsYouTube🦜️🔗LangSmithLangSmith DocsLangServe GitHubTemplates GitHubTemplates HubLangChain HubJS/TS Docs💬SearchGet startedIntroductionInstallationQuickstartSecurityLangChain Expression LanguageGet startedWhy use LCELInterfaceStreamingHow toCookbookLangChain Expression Language (LCEL)ModulesModel I/ORetrievalAgentsChainsMoreLangServeLangSmithLangGraphGet startedIntroductionOn this pageIntroductionLangChain is a framework for developing applications powered by language models. It enables applications that:Are context-aware: connect a language model to sources of context (prompt instructions, few shot examples, content to ground its response in, etc.)Reason: rely on a language model to reason (about how to answer based on provided context, what actions to take, etc.)This framework consists of se

## Text Splitting - Chunking

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=20)
chunks = text_splitter.split_documents(docs)

In [ ]:
chunks[0]

Document(page_content='Introduction | 🦜️🔗 Langchain', metadata={'source': 'https://python.langchain.com/docs/get_started/introduction', 'title': 'Introduction | 🦜️🔗 Langchain', 'description': 'LangChain is a framework for developing applications powered by language models. It enables applications that:', 'language': 'en'})

In [ ]:
chunks[1]

Document(page_content='Skip to main contentDocsUse casesIntegrationsGuidesAPIMorePeopleVersioningChangelogContributingTemplatesCookbooksTutorialsYouTube🦜️🔗LangSmithLangSmith DocsLangServe GitHubTemplates GitHubTemplates HubLangChain HubJS/TS Docs💬SearchGet', metadata={'source': 'https://python.langchain.com/docs/get_started/introduction', 'title': 'Introduction | 🦜️🔗 Langchain', 'description': 'LangChain is a framework for developing applications powered by language models. It enables applications that:', 'language': 'en'})

In [ ]:
chunks[2]

Document(page_content='Docs💬SearchGet startedIntroductionInstallationQuickstartSecurityLangChain Expression LanguageGet startedWhy use LCELInterfaceStreamingHow toCookbookLangChain Expression Language (LCEL)ModulesModel I/ORetrievalAgentsChainsMoreLangServeLangSmithLangGraphGet', metadata={'source': 'https://python.langchain.com/docs/get_started/introduction', 'title': 'Introduction | 🦜️🔗 Langchain', 'description': 'LangChain is a framework for developing applications powered by language models. It enables applications that:', 'language': 'en'})

In [ ]:
chunks[3]

Document(page_content='startedIntroductionOn this pageIntroductionLangChain is a framework for developing applications powered by language models. It enables applications that:Are context-aware: connect a language model to sources of context (prompt instructions, few shot', metadata={'source': 'https://python.langchain.com/docs/get_started/introduction', 'title': 'Introduction | 🦜️🔗 Langchain', 'description': 'LangChain is a framework for developing applications powered by language models. It enables applications that:', 'language': 'en'})

In [ ]:
chunks[4]

Document(page_content='few shot examples, content to ground its response in, etc.)Reason: rely on a language model to reason (about how to answer based on provided context, what actions to take, etc.)This framework consists of several parts.LangChain Libraries: The Python and', metadata={'source': 'https://python.langchain.com/docs/get_started/introduction', 'title': 'Introduction | 🦜️🔗 Langchain', 'description': 'LangChain is a framework for developing applications powered by language models. It enables applications that:', 'language': 'en'})

## Embeddings

In [ ]:
embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=HF_TOKEN, model_name="BAAI/bge-base-en-v1.5"
)

## Vector Store - FAISS or ChromaDB

In [ ]:
vectorstore = Chroma.from_documents(chunks, embeddings)

In [ ]:
vectorstore

In [ ]:
query = "What is Langachain?"
search = vectorstore.similarity_search(query)

In [ ]:
search[0].page_content

'Introduction | 🦜️🔗 Langchain'

## Retriever

In [ ]:
retriever = vectorstore.as_retriever(
    search_type="mmr", #similarity
    search_kwargs={'k': 4}
)

In [ ]:
retriever.get_relevant_documents(query)

[Document(page_content='Introduction | 🦜️🔗 Langchain', metadata={'description': 'LangChain is a framework for developing applications powered by language models. It enables applications that:', 'language': 'en', 'source': 'https://python.langchain.com/docs/get_started/introduction', 'title': 'Introduction | 🦜️🔗 Langchain'}),
 Document(page_content='Templates: A collection of easily deployable reference architectures for a wide variety of tasks.LangServe: A library for deploying LangChain chains as a REST API.LangSmith: A developer platform that lets you debug, test, evaluate, and monitor chains', metadata={'description': 'LangChain is a framework for developing applications powered by language models. It enables applications that:', 'language': 'en', 'source': 'https://python.langchain.com/docs/get_started/introduction', 'title': 'Introduction | 🦜️🔗 Langchain'}),
 Document(page_content="up on our Security best practices to make sure you're developing safely with LangChain.noteThese doc

## Large Language Model - Open Source

In [ ]:
llm = HuggingFaceHub(
    repo_id="huggingfaceh4/zephyr-7b-alpha",
    model_kwargs={"temperature": 0.5, "max_length": 64,"max_new_tokens":512}
)

## Prompt Template and User Input (Augment - Step 2)

In [ ]:
query = "What is Langchain?"

prompt = f"""
 <|system|>
You are an AI assistant that follows instruction extremely well.
Please be truthful and give direct answers
</s>
 <|user|>
 {query}
 </s>
 <|assistant|>
"""

## RAG RetrievalQA chain

In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="refine", retriever=retriever)

In [ ]:
response = qa.run(prompt)

In [ ]:
response

"The original question is as follows: \n <|system|>\nYou are an AI assistant that follows instruction extremely well.\nPlease be truthful and give direct answers\n</s>\n <|user|>\n What is Langchain?\n </s>\n <|assistant|>\n\nWe have provided an existing answer: The original question is as follows: \n <|system|>\nYou are an AI assistant that follows instruction extremely well.\nPlease be truthful and give direct answers\n</s>\n <|user|>\n What is Langchain?\n </s>\n <|assistant|>\n\nWe have provided an existing answer: The original question is as follows: \n <|system|>\nYou are an AI assistant that follows instruction extremely well.\nPlease be truthful and give direct answers\n</s>\n <|user|>\n What is Langchain?\n </s>\n <|assistant|>\n\nWe have provided an existing answer: Context information is below. \n------------\nIntroduction | 🦜️🔗 Langchain\n------------\nGiven the context information and not prior knowledge, answer the question: \n <|system|>\nYou are an AI assistant that fol

## Chain

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import ChatPromptTemplate

In [ ]:
template = """
 <|system|>
You are an AI assistant that follows instruction extremely well.
Please be truthful and give direct answers
</s>
 <|user|>
 {query}
 </s>
 <|assistant|>
"""

In [ ]:
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
rag_chain = (
    {"context": retriever,  "query": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
response = rag_chain.invoke("What is Langchain?")

In [ ]:
print(response)

Human: 
 <|system|>
You are an AI assistant that follows instruction extremely well.
Please be truthful and give direct answers
</s>
 <|user|>
 What is Langchain?
 </s>
 <|assistant|>
Langchain is a Python library for building chatbots, assistants, and other natural language processing applications. It offers a simple and consistent API for working with various text sources and output formats, making it easy to build complex NLP pipelines. Langchain supports a variety of text sources, including web pages, PDFs, and local text files, and output formats, including text, JSON, and CSV. It also offers support for popular NLP frameworks like SpaCy and NLTK, making it a popular choice for building NLP applications in Python.
